## Apartment Review

A simple AI-powered chatbot application that allows users to add and view apartment reviews using natural language. It integrates an LLM for intent understanding, SQLite for data storage, and Gradio for a streaming chat interface, with tool calling and error handling for reliable operation.



In [48]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import sqlite3

In [49]:

MODEL = "gpt-4o-mini"
# load environment variables
load_dotenv(override=True)
API_KEY = os.getenv("OPENAI_API_KEY")
BASE_URL = os.getenv("OPENAI_BASE_URL")

In [62]:
#validate environment variables
if not API_KEY or not BASE_URL:
    raise ValueError(f"{API_KEY} and {BASE_URL} must be set")

# Check the key

if not API_KEY:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not API_KEY.startswith(("sk-proj-", "sk-or-v1-")):
    print("An API key was found, but it doesn't start sk-proj- or sk-or-v1-; please check you're using the right key - see troubleshooting notebook")
elif API_KEY.strip() != API_KEY:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [63]:
client = OpenAI(
    api_key=API_KEY,
    base_url=BASE_URL
)


# connect to the database
DB_FILE = "apartment_reviews.db"
def init_db():
    try:
        conn = sqlite3.connect(DB_FILE)
        cursor = conn.cursor()
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS reviews (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                apartment TEXT NOT NULL,
                reviewer TEXT,
                rating INTEGER,
                comment TEXT,
                created_at DATETIME DEFAULT CURRENT_TIMESTAMP
            )
        """)
        conn.commit()
        #seed data if empty
        cursor.execute("SELECT COUNT(*) FROM reviews")
        count = cursor.fetchone()[0]
        if count == 0:
            seed_reviews(cursor)
        conn.commit()
        conn.close()    
    except Exception as e:
        print(f"Error connecting to the database: {e}")

def seed_reviews(cursor):
    reviews = [
        ("Sunset Apartments", "John", 5, "Amazing place, very clean"),
        ("Sunset Apartments", "Mary", 4, "Nice view and quiet"),
        ("Ocean View Towers", "David", 3, "Good but expensive"),
        ("City Heights", "Alice", 5, "Perfect location"),
        ("City Heights", "Bob", 2, "Too noisy")
    ]

    cursor.executemany("""
        INSERT INTO reviews (apartment, reviewer, rating, comment)
        VALUES (?, ?, ?, ?)
    """, reviews)


In [64]:
def add_review(apartment, reviewer, rating, comment):
    print(apartment, reviewer, rating, comment)
    try:

        if rating < 1 or rating > 5:
            return {"error": "Rating must be between 1 and 5"}

        conn = sqlite3.connect(DB_FILE)
        cursor = conn.cursor()

        cursor.execute("""
            INSERT INTO reviews (apartment, reviewer, rating, comment)
            VALUES (?, ?, ?, ?)
        """, (apartment, reviewer, rating, comment))

        conn.commit()
        conn.close()

        return {"success": True}

    except Exception as e:
        return {"error": str(e)}

Apapa Apartment Tunde 3 Average


In [53]:
def get_reviews(apartment):
    try:

        conn = sqlite3.connect(DB_FILE)
        cursor = conn.cursor()

        cursor.execute("""
            SELECT reviewer, rating, comment
            FROM reviews
            WHERE apartment = ?
        """, (apartment,))

        rows = cursor.fetchall()
        conn.close()

        return {"reviews": rows}

    except Exception as e:
        return {"error": str(e)}

In [41]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "add_review",
            "description": "Add apartment review",
            "parameters": {
                "type": "object",
                "properties": {
                    "apartment": {"type": "string"},
                    "reviewer": {"type": "string"},
                    "rating": {"type": "integer"},
                    "comment": {"type": "string"}
                },
                "required": ["apartment", "reviewer", "rating", "comment"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_reviews",
            "description": "Get apartment reviews",
            "parameters": {
                "type": "object",
                "properties": {
                    "apartment": {"type": "string"}
                },
                "required": ["apartment"]
            }
        }
    }
]

In [ ]:
def execute_tool(name, args):

    try:
        if name == "add_review":
            return add_review(**args)

        elif name == "get_reviews":
            return get_reviews(**args)

        else:
            return {"error": "Unknown tool"}

    except Exception as e:
        return {"error": str(e)}


In [54]:
system_prompt = """
You are an apartment review assistant.

You can:
- Add apartment reviews
- Fetch apartment reviews

Use tools when necessary.
"""

In [60]:
def chat(user_message, history):

    if history is None:
        history = []

    # Start messages with system prompt
    messages = [{"role": "system", "content": system_prompt}]

    # Just extend the OpenAI-style history
    messages.extend(history)

    # Add current user message
    messages.append({"role": "user", "content": user_message})

    try:

        response = client.chat.completions.create(
            model=MODEL,
            messages=messages,
            tools=tools,
            tool_choice="auto"
        )

        msg = response.choices[0].message

        # TOOL CALL
        if msg.tool_calls:

            tool_call = msg.tool_calls[0]

            tool_name = tool_call.function.name
            args = json.loads(tool_call.function.arguments)

            result = execute_tool(tool_name, args)

            messages.append(msg)

            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": json.dumps(result)
            })

            final = client.chat.completions.create(
                model=MODEL,
                messages=messages
            )

            return final.choices[0].message.content

        return msg.content

    except Exception as e:
        return f"Error: {str(e)}"

In [ ]:
init_db()

demo = gr.ChatInterface(
    fn=chat,
    type="messages",
    title="Apartment Review Assistant",
    description="Streaming LLM with SQLite tool calls"
)

demo.launch()

* Running on local URL:  http://127.0.0.1:7870
* To create a public link, set `share=True` in `launch()`.


Santos Villa User 5 Looks good to me
Apapa Apartment Kayode 5 Very conducive
